<a href="https://colab.research.google.com/github/rohaan2614/var_from_scratch/blob/master/autoencoder_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Prep

In [1]:
import numpy as np
import gzip
from urllib import request
from tqdm import tqdm
import os

In [2]:
# Constants
url = "https://ossci-datasets.s3.amazonaws.com/mnist/"
filenames = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',
             't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']

In [3]:
# Create the "data" directory if it doesn't exist
os.makedirs("data", exist_ok=True)

In [4]:
# Download the dataset
for filename in tqdm(filenames, desc="Downloading Dataset"):
    request.urlretrieve(url + filename, os.path.join("data", filename))

In [5]:
# Read dataset
data = []
for filename in os.listdir("data"):
    filepath = os.path.join("data", filename)
    with gzip.open(filepath, 'rb') as f:
        if 'labels' in filename:
            # Load the labels as a one-dimensional array of integers
            data.append(np.frombuffer(f.read(), np.uint8, offset=8))
        else:
            # Load the images as a two-dimensional array of pixels
            data.append(np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28))

In [6]:
print("Data Length: ", len(data), "\ndata: ", data)

Data Length:  4 
data:  [array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([7, 2, 1, ..., 4, 5, 6], dtype=uint8), array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)]


In [7]:
# Split into training and testing sets
X_train, y_train, X_test, y_test = data

In [8]:
print("X_Train: ", X_train)
print("X_Test: ", X_test)

X_Train:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
X_Test:  [7 2 1 ... 4 5 6]


In [9]:
# Normalize the pixel values
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

In [10]:
print("X_Train: ", X_train)
print("X_Test: ", X_test)

X_Train:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
X_Test:  [0.02745098 0.00784314 0.00392157 ... 0.01568628 0.01960784 0.02352941]


In [11]:
print("y_Train: ", y_train)
print("y_Test: ", y_test)

y_Train:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_Test:  [5 0 4 ... 5 6 8]


In [12]:
# Convert labels to integers
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

In [13]:
print("y_Train: ", y_train)
print("y_Test: ", y_test)

y_Train:  [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_Test:  [5 0 4 ... 5 6 8]


### Testing Python Class Extension

In [14]:
class testClass:
  def __init__(self, value):
    self.value = value

  def get_value(self):
    return self.value

In [15]:
test_object = testClass(10)
test_object.get_value()

10

In [16]:
def new_method(self, increment):
  self.value += increment

testClass.increment = new_method

In [17]:
test_object.increment(5)
test_object.get_value()

15

# AutoEncoder

In [18]:
import torch.nn as nn
import graphviz

class AutoEncoder(nn.Module):
    def __init__(self,
                 number_hidden_units : int,
                 encoder : nn.Sequential,
                 decoder : nn.Sequential):
        super().__init__()
        self.num_hidden = number_hidden_units
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def describe(self):
        description = (
            f"AutoEncoder Model\n"
            f"=================\n"
            f"Number of Hidden Units: {self.num_hidden}\n"
            f"Encoder:\n"
            f"{self._describe_sequential(self.encoder)}\n"
            f"Decoder:\n"
            f"{self._describe_sequential(self.decoder)}\n"
        )
        print(description)

    def _describe_sequential(self, sequential):
        layer_descriptions = []
        for i, layer in enumerate(sequential):
            layer_descriptions.append(f"  Layer {i + 1}: {layer}")
        return "\n".join(layer_descriptions)


    def visualize_model(self):
            dot = graphviz.Digraph(format='png')

            # Add nodes for each layer
            dot.node('input', 'Input\n(16)')

            # Encoder layers
            for i, layer in enumerate(self.encoder):
                dot.node(f'encoder_{i}', f'{layer.__class__.__name__}\n({layer.in_features} -> {layer.out_features})')

            # Decoder layers
            for i, layer in enumerate(self.decoder):
                dot.node(f'decoder_{i}', f'{layer.__class__.__name__}\n({layer.in_features} -> {layer.out_features})')

            # Add edges between nodes
            dot.edge('input', f'encoder_0')
            for i in range(len(self.encoder) - 1):
                dot.edge(f'encoder_{i}', f'encoder_{i + 1}')
            dot.edge(f'encoder_{len(self.encoder) - 1}', f'decoder_0')
            for i in range(len(self.decoder) - 1):
                dot.edge(f'decoder_{i}', f'decoder_{i + 1}')

            dot.node('output', 'Output\n(16)')
            dot.edge(f'decoder_{len(self.decoder) - 1}', 'output')

            return dot

In [19]:
encoder = nn.Sequential(
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU())

decoder = nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 16),
        nn.Sigmoid())

# Create AutoEncoder instance
autoencoder = AutoEncoder(number_hidden_units=4, encoder=encoder, decoder=decoder)

# Describe the model
autoencoder.describe()

AutoEncoder Model
Number of Hidden Units: 4
Encoder:
  Layer 1: Linear(in_features=16, out_features=8, bias=True)
  Layer 2: ReLU()
  Layer 3: Linear(in_features=8, out_features=4, bias=True)
  Layer 4: ReLU()
Decoder:
  Layer 1: Linear(in_features=4, out_features=8, bias=True)
  Layer 2: ReLU()
  Layer 3: Linear(in_features=8, out_features=16, bias=True)
  Layer 4: Sigmoid()

